# Dangers of Multiple Comparisons

Testing multiple hypothesis from the same data can be problematic. Exhaustively testing all pairwise relationships between variables in a data set is a commonly used, but generally misleading from of multiple comparisons. The chance of finding false significance, using such a **data dredging** approach, can be surprisingly high. 

In this exercise you will perform multiple comparisons on 20 **identically distributed independent (iid)** variables. Ideally, such tests should not find significant relationships, but the actual result is quite different. 

To get started, execute the code in the cell below to load the required packages. 

In [1]:
import pandas as pd
import numpy as np
import numpy.random as nr
from scipy.stats import ttest_ind, f_oneway
from itertools import product

In this exercise you will apply a t-test to all pairwise combinations of identical Normally distributed variables. In this case, we will create a data set with 20 iid Normal distributions of 1000 samples each. Execute the code in the cell below to find this data and display the mean and variance of each variable.  

In [2]:
ncolumns = 20
nr.seed(234)
normal_vars = nr.normal(size=(1000,ncolumns))
print('The means of the columns are\n', np.mean(normal_vars, axis = 0))
print('\nThe variances of the columns are\n', np.var(normal_vars, axis = 0))

The means of the columns are
 [-1.16191649e-01  2.80829317e-02 -1.78516419e-02 -1.44691489e-02
  3.03718152e-02  1.20007442e-02 -9.58845606e-05  1.98662580e-03
  4.94154934e-02 -4.11640866e-02 -6.32977862e-03 -5.93868192e-02
 -2.56373595e-02  1.43568791e-02 -1.44725765e-02 -1.37023955e-02
  1.80622439e-02  5.87029691e-02 -2.02650514e-02 -1.56346106e-02]

The variances of the columns are
 [0.94834508 1.04744241 1.0258018  0.96977571 1.0089001  1.04113864
 1.00657222 0.99192594 1.04713487 1.04329434 1.04023108 0.96791346
 1.03706907 1.07179865 1.01431404 1.05060289 1.02054329 0.9686211
 1.02810287 0.99521555]


Notice that means and variances are close to 0.0 and 1.0. As expected, there is not much difference between these variables. 

Now for each pair of variables we will compute the t-statistic and p-value and append them to lists.

In [3]:
ttest_results = []
p_values = []
for i,j in product(range(ncolumns),range(ncolumns)):
    if(i != j): # We only want to test between different samples 
        t1, t2 = ttest_ind(normal_vars[:,i], normal_vars[:,j])
        ttest_results.append(t1)
        p_values.append(t2)

How many of these t-tests will show **significance** at the 0.05 cut-off level? There are 380 pairwise combinations, so we expect to find a number of falsely significant test results at this level. To find out, complete and execute the code in the cell below to filter the test results and print those that show significance. 

In [4]:
signifiance_level = 0.05
def find_significant(p_values, ttest_results, signifiance_level):
    n_cases = 0
    for i in range(len(p_values)):
        ##### Add the missing if statement here #############
        if(p_values[i] < signifiance_level): 
            n_cases += 1
            print('t-test with SIGNIFICANT, t-statistic = ', round(ttest_results[i],2), ' and p-value = ', round(p_values[i],4))
    print('\nNumber of falsely significant tests = ', n_cases)        
find_significant(p_values, ttest_results, signifiance_level)        

t-test with SIGNIFICANT, t-statistic =  -3.23  and p-value =  0.0013
t-test with SIGNIFICANT, t-statistic =  -2.21  and p-value =  0.0271
t-test with SIGNIFICANT, t-statistic =  -2.32  and p-value =  0.0204
t-test with SIGNIFICANT, t-statistic =  -3.31  and p-value =  0.0009
t-test with SIGNIFICANT, t-statistic =  -2.87  and p-value =  0.0041
t-test with SIGNIFICANT, t-statistic =  -2.62  and p-value =  0.0087
t-test with SIGNIFICANT, t-statistic =  -2.68  and p-value =  0.0074
t-test with SIGNIFICANT, t-statistic =  -3.71  and p-value =  0.0002
t-test with SIGNIFICANT, t-statistic =  -2.46  and p-value =  0.0139
t-test with SIGNIFICANT, t-statistic =  -2.03  and p-value =  0.0424
t-test with SIGNIFICANT, t-statistic =  -2.9  and p-value =  0.0037
t-test with SIGNIFICANT, t-statistic =  -2.29  and p-value =  0.0218
t-test with SIGNIFICANT, t-statistic =  -2.29  and p-value =  0.0221
t-test with SIGNIFICANT, t-statistic =  -3.02  and p-value =  0.0025
t-test with SIGNIFICANT, t-statisti

Notice the large number of apparently significant tests. Do you trust these results to show any important relationships in the data? 

Can the Bonforoni correction help? Execute the code in the cell below to apply the Bonforoni adjusted significance level to the p-value and ttest data.  

> ### Bonfirroni correction  
> Several adjustments to the multiple comparisons problem have been proposed. In 1979 Holm published a method know as the **Bonfirroni correction**. The adjustment is simple:
$$\alpha_b = \frac{\alpha}{m}\\
with\\ 
m =\ number\ of\ groups$$
> The problem with the Bonfirroni correction is the reduction in power as the  grows smaller. For big data problems with large numbers of groups, this issue can be especially serious. 


In [5]:
signifiance_bonforoni = signifiance_level/380.0
print('With Bonforoni correction the significance level is now = ', signifiance_bonforoni)
find_significant(p_values, ttest_results, signifiance_bonforoni)  

With Bonforoni correction the significance level is now =  0.00013157894736842105
t-test with SIGNIFICANT, t-statistic =  -3.99  and p-value =  0.0001
t-test with SIGNIFICANT, t-statistic =  3.99  and p-value =  0.0001

Number of falsely significant tests =  2


Even with the Bonforoni correction we have some false significance tests, if only just barely!

But, can we detect small effect with Bonforoni correction, as this method significantly reduces power of tests? Execute the code in the cell below, which compares a standard Normal to a Normal with a small mean (effect size), to find out. 

In [6]:
nr.seed(567)
ttest_ind(normal_vars[:,0], nr.normal(loc = 0.01, size=(1000,1)))

Ttest_indResult(statistic=array([-2.49553488]), pvalue=array([0.01265684]))

Given the Bonforoni correction, this difference in means would not be found significant. This illustrates the downside of the correction, which may prevent detection of significant effects, while still finding false significance. 

##### Copyright 2020, Stephen F. Elston. All rights reserved. 